In [ ]:
# Generate Input Document of Survey Questions for LLM API

import pandas as pdas

survey_questions = [
    ('SAMPLESURVEYQUESTION'),
]

#We may need to tweak the survey input prompt text a little bit to ensure we receive back from the model a simple numerical answer to each question. 
def generate_survey_questions(s_question):
    return [
        f'SURVEYPROMPTTEXT: "{s_question}."'
    ]

inputs = []
for s_question in survey_questions:
    inputs.extend(generate_survey_questions(s_question))

pdas.DataFrame(inputs, columns=['Inputs']).to_csv(r"d:\inputs.csv", index=False)

In [ ]:
# GPT API Call

import csv
from openai import OpenAI
import time

# Configuration
client = OpenAI(api_key="APIKEY")
input_file = 'FILEPATH'
output_file = 'outputs.csv'
model = 'MODEL' 
max_tokens = 8
temperature = 1 #We will need to discuss if we'd like to change this parameter

def read_prompts(file_path):
    with open(file_path, newline='', encoding='utf-8', errors='replace') as csvfile:
        reader = csv.DictReader(csvfile)
        return [row['Inputs'] for row in reader]

def write_outputs(prompts, responses, file_path):
    with open(file_path, mode='w', newline='', encoding='utf-8', errors='replace') as csvfile:
        fieldnames = ['Input', 'Output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for prompt, response in zip(prompts, responses):
            writer.writerow({'Input': prompt, 'Output': response})

def call_model(prompt):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error for prompt '{prompt}': {e}")
        return "ERROR"


def main():
    prompts = read_prompts(input_file)
    responses = []

    for i, prompt in enumerate(prompts):
        print(f"Processing prompt {i + 1}/{len(prompts)}")
        response = call_model(prompt)
        responses.append(response)
        time.sleep(2)  # Avoid hitting rate limits

    write_outputs(prompts, responses, output_file)
    print(f"\nAll model responses saved to '{output_file}'.")

if __name__ == '__main__':
    main()

Processing prompt 1/1

All model responses saved to 'outputs.csv'.
